# Sheet properties

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
%matplotlib inline
# produce vector inline graphics
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'svg')
import matplotlib.pyplot as plt

# Start with local

In [ ]:
import sys
from IPython.display import HTML
sys.path.append('/home/rcendre/classification')

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="1";  

# Imports

In [ ]:
import os
import sys
import itertools
import webbrowser
from pathlib import Path
import matplotlib.pyplot as plt
from numpy import array, logspace
from scipy.stats import randint as randint
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.manifold import TSNE
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import davies_bouldin_score
from toolbox.classification.common import Folds, IO, Tools
from toolbox.classification.parameters import Dermatology, Settings
from toolbox.models.builtin import Applications
from toolbox.IO import dermatology
from toolbox.transforms.common import PredictorTransform
from toolbox.transforms.labels import OrderedEncoder
from toolbox.transforms.images import DistributionImageTransform, DWTImageTransform, FourierImageTransform, HaralickImageTransform, SpatialImageTransform
from toolbox.views.common import Views, ViewsTools
from toolbox.views.images import ImagesViews
from IPython.display import display
from toolbox.classification.parameters import Dermatology, Settings, LocalParameters

# Parameters

In [ ]:
# Advanced parameters
use_unknown = False 
validation = 4
settings = Settings.get_default_dermatology()

# Inputs

In [ ]:
inputs = Dermatology.images(modality='Microscopy', data_type=data_type, use_unknown=use_unknown)

In [ ]:
# # Transform groups
# group_encoder = LabelEncoder().fit(array(inputs['ID_Patient'].tolist()))
# Tools.transform(inputs, {'datum': 'ID_Patient'}, group_encoder, 'GroupEncode')
# # Transform labels
# label_encoder = OrderedEncoder().fit(['Normal', 'Benign', 'Malignant'])
# Tools.transform(inputs, {'datum': 'Label'}, label_encoder, 'LabelEncode')
# # Make folds
# Folds.build_group_folds(inputs, {'datum': 'Datum', 'label_encode': 'LabelEncode', 'group': 'GroupEncode'}, validation)

# Feature extraction

In [ ]:
features_file = f'Extraction_Transfer.pickle'

In [ ]:
architectures = [('VGG16', {}), ('InceptionV3', {}), ('InceptionResNetV2', {}), ('ResNet', {})]

In [ ]:
# for architecture in architectures:
#     architecture_name = architecture[0]
#     architecture_additional = architecture[1]
#     # Extracting
#     method = Applications.get_transfer_learning(architecture=architecture_name, pooling='avg', batch_size=1, additional=architecture_additional)
#     method.transform = method.predict_proba
#     Tools.transform(inputs, {'datum': 'Datum'}, method, f'{architecture_name}Avg')

#     # Extracting
#     method = Applications.get_transfer_learning(architecture=architecture_name, pooling='max', batch_size=1, additional=architecture_additional)
#     method.transform = method.predict_proba
#     Tools.transform(inputs, {'datum': 'Datum'}, method, f'{architecture_name}Max')
    
#     # Save    
#     IO.save(inputs, features_file)

# Features

In [ ]:
inputs = IO.load(features_file)
for architecture in architectures: 
    extractor_name, extractor_process = architecture
    # PCA + TSNE
    figure = Views.projection(inputs, {'datum': f'{extractor_name}Avg', 'label': 'Label'}, settings);
    figure.savefig(f"{extractor_name}Avg.png", dpi=300)
    
    figure = Views.projection(inputs, {'datum': f'{extractor_name}Max', 'label': 'Label'}, settings);
    figure.savefig(f"{extractor_name}Max.png", dpi=300)